<a href="https://colab.research.google.com/github/Tomawock/NLP_Attack/blob/main/model/ATE_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [145]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [146]:
import numpy
import pandas as pd

Load Dataset

In [147]:
test = pd.read_json('/content/drive/Shareddrives/Deep Learning/datasets/ATE_ABSITA/ATE_ABSITA_test_set/ate_absita_gold.ndjson'
                       , lines=True)

In [148]:
train = pd.read_json('/content/drive/Shareddrives/Deep Learning/datasets/ATE_ABSITA/ATE_ABSITA_training_set/ate_absita_training.ndjson'
                       , lines=True)

In [149]:
print(test.iloc[12].id_sentence)

00a7a15c-a281-43d3-bbdb-c718f7874ca8


In [150]:
train.head(3)

,sentence,id_sentence,score,polarities,aspects_position,aspects
0,"Ottimo prodotto di marca, la qualità é veramen...",4b7254a1-3f31-4143-ab22-a8558aa4a73b,5,"[[0, 0], [0, 1], [1, 0]]","[[120, 142], [71, 79], [29, 36]]","[provvisto di una tasca, capiente, qualità]"
1,Ottimo rasoio dal semplice utilizzo. Rade molt...,4b74d99d-891f-4526-bbd3-549fa244cd1c,5,"[[1, 0], [1, 0], [1, 0], [1, 0]]","[[18, 26], [37, 41], [79, 86], [99, 105]]","[semplice, Rade, Pratico, pulire]"
2,Un quarto delle dimensioni dello Show original...,4b7ff44f-fa9f-4ef0-97c8-e295e70ccc9b,5,"[[1, 0], [1, 0], [1, 0], [0, 0]]","[[118, 132], [51, 62], [65, 70], [16, 26]]","[modalità notte, prestazioni, suono, dimensioni]"


In [151]:
i=train[train.sentence == test.iloc[1123].sentence]
print(i)

Empty DataFrame
Columns: [sentence, id_sentence, score, polarities, aspects_position, aspects]
Index: []


In [152]:
print(test.columns)
print(train.columns)

Index(['sentence', 'id_sentence', 'score', 'polarities', 'aspects_position',
       'aspects'],
      dtype='object')
Index(['sentence', 'id_sentence', 'score', 'polarities', 'aspects_position',
       'aspects'],
      dtype='object')


In [153]:
train.drop(columns=['id_sentence','polarities','aspects_position','aspects'], inplace=True)
test.drop(columns=['id_sentence','polarities','aspects_position','aspects'], inplace=True)
print(f'Contains {len(train)} sentences')
print(f'Contains {len(test)} sentences')

Contains 3054 sentences
Contains 1200 sentences


In [154]:
dataset = pd.concat([train,test])

In [161]:
train["review_type"] = train["score"].apply(lambda x: "neg" if x < 5 else "pos")
dataset["review_type"] = dataset["score"].apply(lambda x: "neg" if x < 5 else "pos")

In [162]:
print(f'TRAIN::\n{train.review_type.value_counts()}')
print(f'COMPLETO::\n{dataset.review_type.value_counts()}')

TRAIN::
pos    2150
neg     904
Name: review_type, dtype: int64
COMPLETO::
pos    3007
neg    1247
Name: review_type, dtype: int64


**Necessaria Augemntation**

In [164]:
train.drop(columns=['score'], inplace=True)

In [ ]:
!pip install tensorflow_text

In [188]:
import tensorflow_hub as hub
import numpy as np
import tensorflow_text
from tensorflow import keras
import tensorflow as tf

In [178]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

In [ ]:
tf.reshape(embed('ciao sono pippo'),[1,512,1])

In [179]:
from sklearn.preprocessing import OneHotEncoder

type_one_hot = OneHotEncoder(sparse=False).fit_transform(
  train.review_type.to_numpy().reshape(-1, 1)
)

Trasforma da pos, neg a [0,1], [1,0]

In [183]:
type_one_hot

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [0., 1.],
       [0., 1.]])

In [181]:
train.review_type

0       pos
1       pos
2       pos
3       pos
4       pos
       ... 
3049    neg
3050    pos
3051    neg
3052    pos
3053    pos
Name: review_type, Length: 3054, dtype: object

In [238]:
def UniversalEmbedding(x):
    return tf.reshape(embed(tf.squeeze(tf.cast(x, tf.string))),[1,512,1])#, 
    	#signature="default", as_dict=True)["default"]

In [199]:
tf.version

<module 'tensorflow._api.v2.version' from '/usr/local/lib/python3.6/dist-packages/tensorflow/_api/v2/version/__init__.py'>

In [ ]:
# define the LSTM model
model = Sequential()
model.add(keras.layers.Input(shape=(1,), dtype=tf.string))
model.add(keras.layers.Lambda(UniversalEmbedding))
model.add(keras.layers.LSTM(128))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(0.001), metrics=['accuracy'])
model.summary()

In [249]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(train['sentence'], type_one_hot, epochs=3, batch_size=20, callbacks=callbacks_list)

Epoch 1/3


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:437: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradients/EncoderTransformer/EmbeddingLookup/EmbeddingLookupUnique/GatherV2_grad/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradients/EncoderTransformer/EmbeddingLookup/EmbeddingLookupUnique/GatherV2_grad/Reshape:0", shape=(None, 512), dtype=float32), dense_shape=Tensor("gradients/EncoderTransformer/EmbeddingLookup/EmbeddingLookupUnique/GatherV2_grad/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)


InvalidArgumentError: ignored

(3054,)